In [7]:
search_for = 99
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.003860950469970703
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 75
all cases: 177332155
type: [1, 1, 1, 1, 99] 99
cases of this type: 96059601
10000 25.184566694160203
20000 26.480038031587668
30000 26.522529941278
40000 115.47816754567653
50000 103.46708530306634
60000 93.24393542523492
70000 96.67933290932358
80000 99.78576086732276
90000 102.01373188195004
100000 100.3671893727089
110000 91.90252353556502
120000 98.39039381329324
130000 96.26906864966651
140000 97.44590860952033
150000 103.42165239549789
160000 100.78193755010605
170000 96.9409907007376
180000 90.05425055304491
190000 100.01687815978745
200000 90.18000254600412
210000 93.08645610267368
220000 100.83125110907227
230000 89.8263788679867
240000 102.21702224725074
250000 111.52794353638234
260000 104.05066478445013
270000 91.85150628642953
280000 93.79174972661475
290000 94.61653815286697
300000 99.0988079421979
310000 91.54394605698664
32

3110000 82.47285267456384
3120000 90.09990748490773
3130000 86.52692027474524
3140000 95.87707735352463
3150000 88.4700580432727
3160000 103.65294176372329
3170000 101.80324712507708
3180000 97.41687518084387
3190000 97.76494366873453
3200000 85.03656616364502
3210000 86.69181892655389
3220000 91.02074978553443
3230000 87.39027675131997
3240000 84.57462863212811
3250000 79.11095478162905
3260000 89.36127286655075
3270000 80.6966283074267
3280000 77.34728710752924
3290000 87.84691558322808
3300000 86.62078076608849
3310000 92.21564707411676
3320000 90.04872889829646
3330000 88.11548107896496
3340000 88.99617968297015
3350000 88.38244901164737
3360000 92.72970704445665
3370000 102.27547822636704
3380000 26.34131289249814
3390000 12.263818678868546
3400000 14.699020567418636
3410000 13.594126731405556
3420000 14.373485875863995
3430000 111.4472208937045
3440000 92.66472631647483
3450000 79.65597504832247
3460000 87.46588830206007
3470000 74.81988791193518
3480000 85.71250046713318
3490000

6260000 75.6444415273335
6270000 80.90571689142153
6280000 81.98646396340548
6290000 33.47010247634209
6300000 13.609099147340656
6310000 12.235668777157366
6320000 13.48799342173189
6330000 9.786589632855224
6340000 86.83262524566064
6350000 72.97815837783743
6360000 76.20800587324119
6370000 69.1968670374793
6380000 85.5581678506715
6390000 77.91614014464248
6400000 82.9697097774001
6410000 92.06517265319124
6420000 79.95727131057804
6430000 84.53026886550526
6440000 87.46284787651688
6450000 79.4324684890652
6460000 84.23838427827836
6470000 81.47399814335894
6480000 85.30344906847928
6490000 75.07753866179456
6500000 78.79051970237198
6510000 89.33248567135715
6520000 89.11375526091834
6530000 84.18622289657822
6540000 91.51485817025106
6550000 85.80690751658588
6560000 76.96406828639971
6570000 83.7307903472854
6580000 85.13837010207901
6590000 80.65591232421795
6600000 88.77135842059126
6610000 74.96008343020408
6620000 83.97594900576233
6630000 70.00087765224944
6640000 83.10228

9420000 95.09686379162538
9430000 75.63232242013544
9440000 77.59422551545902
9450000 82.61603378675825
9460000 80.10890780189307
9470000 75.32783175655743
9480000 84.62628453808685
9490000 74.90186067194482
9500000 92.07974021087759
9510000 87.71103508058074
9520000 90.87643320298879
9530000 82.04275420956083
9540000 87.14599588856267
9550000 84.5854166404585
9560000 75.69167996643431
9570000 79.05819846021076
9580000 93.91612019166112
9590000 60.38581066641149
9600000 32.30869420672655
9610000 31.83164854108542
9620000 28.873877119296026
9630000 31.7766219449387
9640000 69.10601323649735
9650000 91.74305492694971
9660000 89.02574332954045
9670000 74.83147890315085
9680000 80.54090679488453
9690000 84.22646645780256
9700000 96.28813205351352
9710000 88.83691543518154
9720000 94.81286581540697
9730000 94.70741813286097
9740000 87.45380108645655
9750000 92.17235035075974
9760000 87.34874640349318
9770000 95.27161090342544
9780000 106.50159877820171
9790000 36.3940631058982
9800000 27.49

12490000 32.05982467847228
12500000 29.71317195324739
12510000 30.801466421599155
12520000 65.73396318471342
12530000 90.70854880388276
12540000 77.9770999294474
12550000 75.82831021854668
12560000 89.25802774743978
12570000 84.99389013356809
12580000 94.8057399493733
12590000 87.47298011228597
12600000 84.31483250559891
12610000 91.52423308941195
12620000 90.09671606845329
12630000 88.73698038837985
12640000 85.973185920056
12650000 82.03915265110702
12660000 75.80072751458586
12670000 81.01896625775456
12680000 88.90279265043986
12690000 79.70650633446087
12700000 75.26097619256895
12710000 86.42419619382329
12720000 99.76617730451358
12730000 24.835263461808363
12740000 26.6052310445604
12750000 21.398148180200558
12760000 24.769141401670577
12770000 27.67001053606325
12780000 84.85864767341177
12790000 87.05801969938318
12800000 72.42433447602781
12810000 93.51299477685602
12820000 86.19835761818696
12830000 87.99380361353738
12840000 96.12469630046047
12850000 92.32212764962435
12

...

165990000 1.728534246349136
166000000 1.6366039485539992
166010000 1.7289793785429335
166020000 1.7165955662479995
166030000 1.7380499252727626
166040000 1.9145794392532776
166050000 1.7567972760882975
166060000 1.7039268896271784
166070000 1.6777630737814968
166080000 1.6532917189576888
166090000 1.6379098123449087
166100000 1.7283917387275896
166110000 1.787862268662519
166120000 1.7620730678880216
166130000 1.632254261664715
166140000 1.7973470553872652
166150000 1.9152514403755463
166160000 1.6844270148821674
166170000 1.6806355786972904
166180000 1.6256201660551628
166190000 1.5482842375151316
166200000 1.615760112167961
166210000 1.6112893896022438
166220000 1.6852291567499769
166230000 1.731816018014716
166240000 1.61650204747051
166250000 1.6963652765567432
166260000 1.8520690439409346
166270000 1.7523279085061947
166280000 1.687555669667224
166290000 1.688484742256039
166300000 1.6211734952795507
166310000 1.6605372657149369
166320000 1.6823987126264968
166330000 1.77260532423

168750000 0.6780519944792655
168760000 0.6479173180200656
168770000 0.646319524007572
168780000 0.6709248654279444
168790000 0.6280595557587345
168800000 0.6510161275634833
168810000 1.7904054460862608
168820000 2.1105474897077676
168830000 2.16949416720791
168840000 2.2183260567457808
168850000 2.141409763124287
168860000 2.162681881031328
168870000 2.1567807515409254
168880000 2.069737861639261
168890000 2.1138944232929084
168900000 2.032706258627243
168910000 2.1690785630413894
168920000 2.125127691677676
168930000 2.107503089886639
168940000 1.9630800977189342
168950000 1.9946974937278694
168960000 1.919406861127085
168970000 1.9693021056374906
best so far: 0
type: [1, 1, 11, 3, 3] 99
cases of this type: 264627
168980000 2.0085274439475937
168990000 1.4196652545134556
169000000 1.3643642758998273
169010000 1.3432670282839405
169020000 1.3400403856990404
169030000 1.3825003585948548
169040000 1.2980862603666716
169050000 1.3754088501123256
169060000 1.3415579910736282
169070000 1.37

171460000 0.42418511965165534
171470000 0.44978932991511295
171480000 0.4529242150786188
171490000 0.45856319849242766
171500000 0.4453909059152007
171510000 1.1325095692326956
171520000 2.668679893827438
171530000 2.1284756720707483
171540000 2.562750066812237
171550000 1.8466597754618523
171560000 0.7437800729735029
171570000 0.5238113568959964
171580000 1.0194467383090655
171590000 2.7615493371405204
171600000 2.1949835198961365
171610000 2.382341132389307
171620000 2.851466879958742
171630000 2.0121495368438627
171640000 1.8143291916850208
171650000 2.091102116545869
171660000 1.1248979744678074
171670000 0.5777097820350527
171680000 0.567624169769254
171690000 0.9449996769296791
171700000 2.1428460070759057
171710000 1.8024184338031544
171720000 2.049967645936145
171730000 2.1862507132294775
171740000 1.5855265793122848
171750000 1.621114236626294
171760000 1.3048759661247333
171770000 0.9774135723793507
171780000 0.44486149625036453
171790000 0.6353380269581079
171800000 0.879375

174250000 0.3363641230333845
174260000 0.4559179813547267
174270000 0.4075704973761241
174280000 0.4754809702604016
174290000 0.4071646180699931
174300000 0.44664941897200217
174310000 0.4187591349583864
174320000 0.4871892363991009
174330000 0.4313844932687613
174340000 0.42884823051194343
174350000 0.3676608711815542
174360000 0.4466682245611813
174370000 0.39149031661182637
174380000 0.41258844407204126
174390000 0.37977289251794416
174400000 0.43500106060594323
174410000 0.415141645190484
174420000 0.4018722738473614
174430000 0.39768237314194443
174440000 0.44073372839679326
174450000 0.4315679359502594
174460000 0.4150861363679171
174470000 0.3777901182656818
174480000 0.3903404736799995
174490000 0.3480336648711562
174500000 0.4242303606997596
174510000 0.40699863745070164
174520000 0.40815987970282636
best so far: 0
type: [1, 3, 1, 33, 1] 99
cases of this type: 107811
174530000 0.26016464133024214
174540000 0.1743519716893633
174550000 0.18420841276248295
174560000 0.1859565825

176560000 0.23290311337292197
176570000 0.09953052943514452
176580000 0.0581012169647879
176590000 0.0958445505180955
176600000 0.10872117145313157
176610000 0.08765682724687787
176620000 0.06205699606170257
176630000 0.1063994404077861
176640000 0.1193876574058334
176650000 0.06104785561054945
176660000 0.08156095476716756
176670000 0.10597012919773659
176680000 0.0882991376175483
176690000 0.053309350968268186
176700000 0.0668422218257851
176710000 0.09530053880522649
176720000 0.0673266355569164
176730000 0.05424103547175725
176740000 0.08971337269067765
176750000 0.10150603618274133
176760000 0.048010212844510874
176770000 0.08301083622167509
176780000 0.07017593345463276
176790000 0.07378754368460841
176800000 0.04774468447605769
176810000 0.09540763193147049
176820000 0.07775570315288172
176830000 0.04072484047681093
176840000 0.054435299919313854
176850000 0.07916259031746121
176860000 0.05366614626010258
176870000 0.040702111081414756
176880000 0.05817642584416602
176890000 0.0